In [12]:
import pandas as pd
import numpy as np
import random

In [13]:
df = pd.read_csv('movies_with_test.csv')

In [14]:
df.shape

(199, 2)

In [15]:
def create_shingles(text, min_length=1, max_length=None):
  tags = [tag.strip() for tag in text.split(',')]  # Tách các tag riêng biệt

  all_shingles = set()
  for tag in tags:
      words = tag.split()  # Tách tag thành các từ
      if max_length is None:
          max_length = max(len(word) for word in words)
      for word in words:
          for i in range(min_length, min(max_length, len(word)) + 1):
              all_shingles.add(word[:i])
  return all_shingles

df['shingles'] = df['genres'].astype(str).apply(create_shingles)

In [16]:
df['shingles']

0                      {Comedy, Comed, C, Come, Co, Com}
1                              {Dram, Dra, Drama, Dr, D}
2      {A, Adu, Comed, C, Come, Adul, Co, Com, Ad, Ad...
3      {Fam, Animat, Short, Shor, Anima, Animatio, An...
4                              {Dram, Dra, Drama, Dr, D}
                             ...                        
194                            {Dram, Dra, Drama, Dr, D}
195                            {Dram, Dra, Drama, Dr, D}
196    {Animat, Adventur, Anima, Animatio, Adve, An, ...
197                    {Comedy, Comed, C, Come, Co, Com}
198    {Animat, Adventur, Anima, Animatio, Adve, An, ...
Name: shingles, Length: 199, dtype: object

In [17]:
def create_hash_functions(num_hash_functions, max_val):
    hash_functions = []
    for _ in range(num_hash_functions):
        a = random.randint(1, max_val)
        b = random.randint(0, max_val)
        hash_functions.append(lambda x, a=a, b=b: (a * x + b) % max_val)
    return a,b

def minhash_signature(shingles, hash_functions):
    signature = [float('inf')] * len(hash_functions) 
    for shingle in shingles:
        shingle_hash = hash(shingle)
        for i, hash_func in enumerate(hash_functions):
            signature[i] = min(signature[i], hash_func(shingle_hash))
    return signature

all_shingles = set()
for shingles_list in df['shingles']:
    all_shingles.update(shingles_list)

print (create_hash_functions(10, 11))
# Create 10 hash functions
# hash_functions = create_hash_functions(10, len(all_shingles))

# # Apply minhash_signature to each row in the `shingles` column
# df['minhash_signature'] = df['shingles'].apply(lambda x: minhash_signature(x, hash_functions))


(10, 0)


In [18]:
df.head(20)

,originalTitle,genres,shingles
0,Episode #1.106,Comedy,"{Comedy, Comed, C, Come, Co, Com}"
1,Fagr Yom gedid,Drama,"{Dram, Dra, Drama, Dr, D}"
2,Denim,"Adult,Comedy","{A, Adu, Comed, C, Come, Adul, Co, Com, Ad, Ad..."
3,The Millionaire,"Animation,Family,Short","{Fam, Animat, Short, Shor, Anima, Animatio, An..."
4,The Pigman's Protege,Drama,"{Dram, Dra, Drama, Dr, D}"
5,Der Hausmeister,Adult,"{A, Adu, Adul, Ad, Adult}"
6,Episode #1.1,Comedy,"{Comedy, Comed, C, Come, Co, Com}"
7,The Ixtafa Affair,"Action,Adventure,Sci-Fi","{A, Act, Advent, Sci-, Adve, Sci, Acti, Ac, Ac..."
8,Flag Wars Community Forum,Documentary,"{Documenta, Docu, D, Document, Documen, Docum,..."
9,Episode #1.4,Drama,"{Dram, Dra, Drama, Dr, D}"


In [19]:
class LSH:
    def __init__(self, num_bands, num_rows_per_band):
        self.num_bands = num_bands
        self.num_rows_per_band = num_rows_per_band
        self.buckets = {}  # Lưu trữ các nhóm (bucket)

    def hash_function(self, signature, band_index):
        """
        Hàm băm đơn giản để ánh xạ một phần của chữ ký MinHash vào một nhóm.
        """
        start_row = band_index * self.num_rows_per_band
        end_row = start_row + self.num_rows_per_band
        band_signature = tuple(signature[start_row:end_row])
        return hash(band_signature)

    def insert(self, key, signature):
        """
        Chèn một mục (key) và chữ ký MinHash của nó vào các nhóm LSH.
        """
        for band_index in range(self.num_bands):
            bucket_id = self.hash_function(signature, band_index)
            if bucket_id not in self.buckets:
                self.buckets[bucket_id] = []
            self.buckets[bucket_id].append(key)

    def movie_similar(self, query_signature):
        """
        Truy vấn các mục có khả năng tương tự dựa trên chữ ký MinHash của truy vấn.
        """
        candidates = set()
        for band_index in range(self.num_bands):
            bucket_id = self.hash_function(query_signature, band_index)
            if bucket_id in self.buckets:
                candidates.update(self.buckets[bucket_id])
        return candidates
    def movie_query(self, tag_query):
        pass
        


In [20]:
# Định nghĩa hàm tính độ tương đồng Jaccard
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

In [21]:
# Khởi tạo LSH với số lượng bands và số hàng mỗi band
lsh = LSH(num_bands=5, num_rows_per_band=2)

# Chèn các bộ phim vào LSH
for index, row in df.iterrows():
    lsh.insert(row['originalTitle'], row['minhash_signature'])

# Truy vấn phim tương tự
query_movie = 'The Millionaire'
query_signature = df[df['originalTitle'] == query_movie]['minhash_signature'].iloc[0]
candidate_movies = lsh.movie_similar(query_signature)

# Kiểm tra danh sách phim ứng viên
print("Danh sách phim ứng viên:", candidate_movies)

# Tính độ tương đồng Jaccard cho các phim ứng viên
similarities = []
for candidate_movie in candidate_movies:
    if candidate_movie in df['originalTitle'].values:
        candidate_signature = df[df['originalTitle'] == candidate_movie]['minhash_signature'].iloc[0]
        raw_similarity = jaccard_similarity(set(query_signature), set(candidate_signature))
        # Đảm bảo rằng giá trị của len(hash_functions) không gây chia cho 0 hoặc gây lỗi
        normalized_similarity = raw_similarity / len(hash_functions)
        print(f"Movie: {candidate_movie}, Raw Similarity: {raw_similarity}, Normalized Similarity: {normalized_similarity}")
        similarities.append((candidate_movie, normalized_similarity))
    else:
        print(f"Chữ ký MinHash không tìm thấy cho phim: {candidate_movie}")

KeyError: 'minhash_signature'

In [22]:
# Lọc phim dựa trên ngưỡng độ tương đồng
threshold = 0.05
filtered_movies = [(movie, sim) for movie, sim in similarities if sim > threshold]

# In kết quả phim tương tự
for movie, similarity in filtered_movies:
    if movie != query_movie:  # Loại trừ phim truy vấn
        print(f"Movie: {movie}, Jaccard Similarity: {similarity}")

print("Danh sách phim lọc:", filtered_movies)

NameError: name 'similarities' is not defined